In [1]:
import cantera as ct
import numpy as np
import pandas as pd
import csv
import sys
import os

print("Running Cantera Version: " + str(ct.__version__ ))

Running Cantera Version: 2.5.1


In [25]:
To = 298
Po = ct.one_atm
path_to_mech = '/work/westgroup/nora/Code/projects/PFAS/simulations/WPI_flamespeeds/models/NUIG_Brown5_edited_spc_names.yaml'
gas_full = ct.Solution(path_to_mech)
#####
# use this toggle to use either the full mechanism (True) or a smaller version (False)
FullMech = False
#FullMech = True

if FullMech==False:
    
    #exclude = ['CH2CCHCH2CHCH2'] #add specific species here. (for illustration purposes, since the Max_N_oxygen below would eliminate it anyway)
    max_N_carbon = 4
    max_N_oxygen = 3
    
    all_species = ct.Species.listFromFile(path_to_mech)
    species = []
    # Filter species
    for S in all_species:
#         if S.name in exclude:
#             print( "excluding %s"%(S.name) )
#             continue #skip this species
        comp = S.composition
        if S.name in ['C6H9', 'C6H10']:
            continue
        if 'C' in comp and comp['C']> max_N_carbon:
            print( "excluding %s"%(S.name) )
            continue
        if 'O' in comp and comp['O']> max_N_oxygen:
            print( "excluding %s"%(S.name) )
            continue            
#         if 'He' in comp: # Exclude Helium
#             print( "excluding %s"%(S.name) )
#             continue
#         if 'Ar' in comp: # Exclude Argon
#             print( "excluding %s"%(S.name) )
#             continue
#         if 'Kr' in comp: # Exclude Krypton
#             print( "excluding %s"%(S.name) )
#             continue   
        #print S.name    
        species.append(S)

    species_names = {S.name for S in species}
    # Filter reactions, keeping only those that only involve the selected species
    all_reactions = gas_full.reactions()
    reactions = []

    for R in all_reactions:
        if not all(reactant in species_names for reactant in R.reactants):
            continue
        if not all(product in species_names for product in R.products):
            continue
        reactions.append(R)

    gas_small = ct.Solution(thermo='IdealGas', kinetics='GasKinetics',
                       species=species, reactions=reactions)
    gas = gas_small    
else:
    gas = gas_full
    
print( "final mechanism has %d species and %d reactions"%(gas.n_species, gas.n_reactions) )
print(path_to_mech)

For species OHV, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  53.62056162666667
	Value computed using high-temperature polynomial: 53.5841554314

For species CHV, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  107.5046684
	Value computed using high-temperature polynomial: 107.34847808033332

For species KCHOCH2CO3H, discontinuity in cp/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  22.934254100000004
	Value computed using high-temperature polynomial: 23.200262099999996

excluding O2CH2OCH2O2H
excluding HO2CH2OCHO
excluding C3H6OOH1-2O2
excluding C3H6OOH1-3O2
excluding C3H6OOH2-1O2
excluding C3H51-2,3OOH
excluding C3H52-1,3OOH
excluding QC3H5OHPO2
excluding C3H4OH1KT2-3
excluding TQC3H5OHIO2
excluding TQC3H5OHIQ-I
excluding TQC3H5OHIQ-P
excluding COHOOHCY(COC)
excluding OHCY(COCC)OOH
excluding IQC3H5OHPJO2
excluding IQC3H5OTQ-I
excluding IQC3H5OHQ-SJ


In [26]:
mole_fractions = list(np.linspace(0.1, 0.4, 40))   #mole fraction of propane
results = {}

pos_in_list = 2

x = mole_fractions[pos_in_list]
phi=x/.2
print(phi)

0.5769230769230769


In [31]:
mole_frac_dict = {'C3H8': x, 'O2':1, 'N2':3.76} 
gas.TPX = To, Po, mole_frac_dict
width = 0.08
flame = ct.FreeFlame(gas, width=width)
#Set maxiumum number of grid points to be very high (otherwise default is 1000)
flame.set_max_grid_points(flame.domains[flame.domain_index("flame")], 1e4)
#flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1)
#flame.set_refine_criteria(ratio=5, slope=0.2, curve=0.3) 
flame.set_refine_criteria(ratio=10, slope=0.8, curve=0.8) 
flame.max_time_step_count = 2600
loglevel = 1 
if pos_in_list!=0:
    previous_phi = (mole_fractions[pos_in_list-1])/.2
    d = f'{previous_phi}_edited_names.csv'
    print(f'Looking for a data file that matches {d}')
    if os.path.exists(d):  
        arr2 = ct.SolutionArray(gas)
        arr2.read_csv(d)
        flame.set_initial_guess(data=arr2)
        print('success')

Looking for a data file that matches 0.5384615384615384_edited_names.csv
success


In [ ]:
flame.solve(loglevel=loglevel, auto=False) #turning off auto=False
Su = flame.velocity[0]
results[phi] = Su
print(f' at {phi} phi, speed is {Su}')


..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      1.78e-06      7.863
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps   